In [1]:
import numpy as np
from numpy import linalg
import sklearn
from sklearn import decomposition
from sklearn import datasets

In [2]:
print('Numpy version {}.'.format(np.__version__))
print('Sklearn version {}.'.format(sklearn.__version__))

Numpy version 1.13.1.
Sklearn version 0.19.0.


---

# PCA Walkthrough (whiteboard)

In [3]:
# generate data
my_data = np.array([1,4,3,2]).reshape(2,2)
my_data

array([[1, 4],
       [3, 2]])

In [4]:
# matrix of means
mu_data = my_data.mean(axis=0)
mu_data

array([ 2.,  3.])

In [5]:
# covariance matrix
np.cov(my_data)

array([[ 4.5, -1.5],
       [-1.5,  0.5]])

In [6]:
eig_vals, eig_vecs = linalg.eigh(np.cov(my_data))
eig_vals

array([  5.55111512e-17,   5.00000000e+00])

In [7]:
eig_vecs

array([[-0.31622777, -0.9486833 ],
       [-0.9486833 ,  0.31622777]])

---

# PCA Algorithm

PCA requires several steps:

1) demean columns  
2) calculate covariance matrix  
3) calculate eigenvalues and eigenvectors  
4) sort eigenvectors and eigenvalues in descending order  
5) transform original dataset using sorted eigenvectors

## Get Data

In [8]:
iris = datasets.load_iris()
X = iris.data
X[:5]

array([[ 5.1,  3.5,  1.4,  0.2],
       [ 4.9,  3. ,  1.4,  0.2],
       [ 4.7,  3.2,  1.3,  0.2],
       [ 4.6,  3.1,  1.5,  0.2],
       [ 5. ,  3.6,  1.4,  0.2]])

## From Scratch

#### Step 1 - demean columns

In [9]:
mu = X.mean(axis=0)
mu

array([ 5.84333333,  3.054     ,  3.75866667,  1.19866667])

In [10]:
X_demean = X - mu
X_demean[:5]

array([[-0.74333333,  0.446     , -2.35866667, -0.99866667],
       [-0.94333333, -0.054     , -2.35866667, -0.99866667],
       [-1.14333333,  0.146     , -2.45866667, -0.99866667],
       [-1.24333333,  0.046     , -2.25866667, -0.99866667],
       [-0.84333333,  0.546     , -2.35866667, -0.99866667]])

#### Steps #2 - calculate covariance matrix

In [11]:
X_demean.shape

(150, 4)

In [12]:
# calculate covariance matrix (automatically demeans columns)
X_cov = np.cov(X, rowvar=False)

In [13]:
X_cov

array([[ 0.68569351, -0.03926846,  1.27368233,  0.5169038 ],
       [-0.03926846,  0.18800403, -0.32171275, -0.11798121],
       [ 1.27368233, -0.32171275,  3.11317942,  1.29638747],
       [ 0.5169038 , -0.11798121,  1.29638747,  0.58241432]])

#### Step #3 - calculuate eigenvalues and eigenvectors

In [14]:
# calculate eigenvectors and eigenvalues
eigenvals, eigenvecs = linalg.eigh(X_cov)
eigenvals, eigenvecs

(array([ 0.02368303,  0.07852391,  0.24224357,  4.22484077]),
 array([[ 0.31725455,  0.58099728,  0.65653988, -0.36158968],
        [-0.32409435, -0.59641809,  0.72971237,  0.08226889],
        [-0.47971899, -0.07252408, -0.1757674 , -0.85657211],
        [ 0.75112056, -0.54906091, -0.07470647, -0.35884393]]))

#### Step #4 - sort eigenvalues and eigenvectors

In [39]:
def get_principle_comp(eig_vals, eig_vecs, dimensions):

    # We sort the eigvals and return the indices for the
    # ones we want to include (specified by "dimensions" paramater)
    eigval_max = np.argsort(-eig_vals)[:dimensions]
    eigvec_max = eig_vecs[:, eigval_max]
    return eigvec_max, eigval_max

In [40]:
# multiply by -1 (since signs of eigenvectors can be + or -)
principal_components, eigval_idx = get_principle_comp(eigenvals, eigenvecs, 3)

In [41]:
-principal_components.T

array([[ 0.36158968, -0.08226889,  0.85657211,  0.35884393],
       [-0.65653988, -0.72971237,  0.1757674 ,  0.07470647],
       [-0.58099728,  0.59641809,  0.07252408,  0.54906091]])

In [54]:
explained_variance = []
for i in eigenvals[eigval_idx]:
    explained_variance.append(i / np.sum(eigenvals))
explained_variance

[0.92461620717426851, 0.053015567850534961, 0.017185139525006828]

#### Step #5 - transform original dataset using sorted eigenvectors

In [18]:
X_transformed_scratch = X_demean.dot(principal_components)
X_transformed_scratch[:10]

array([[-2.68420713, -0.32660731, -0.02151184],
       [-2.71539062,  0.16955685, -0.20352143],
       [-2.88981954,  0.13734561,  0.02470924],
       [-2.7464372 ,  0.31112432,  0.03767198],
       [-2.72859298, -0.33392456,  0.0962297 ],
       [-2.27989736, -0.74778271,  0.17432562],
       [-2.82089068,  0.08210451,  0.26425109],
       [-2.62648199, -0.17040535, -0.01580151],
       [-2.88795857,  0.57079803,  0.02733541],
       [-2.67384469,  0.1066917 , -0.1915333 ]])

---

## Sklearn PCA

In [19]:
pca = decomposition.PCA(n_components=3, random_state=42)
pca.fit(X)
X_transformed_sklearn = pca.transform(X)

In [20]:
pca.components_

array([[ 0.36158968, -0.08226889,  0.85657211,  0.35884393],
       [ 0.65653988,  0.72971237, -0.1757674 , -0.07470647],
       [-0.58099728,  0.59641809,  0.07252408,  0.54906091]])

In [21]:
pca.explained_variance_ratio_

array([ 0.92461621,  0.05301557,  0.01718514])

In [22]:
X_transformed_sklearn[:10]

array([[-2.68420713,  0.32660731, -0.02151184],
       [-2.71539062, -0.16955685, -0.20352143],
       [-2.88981954, -0.13734561,  0.02470924],
       [-2.7464372 , -0.31112432,  0.03767198],
       [-2.72859298,  0.33392456,  0.0962297 ],
       [-2.27989736,  0.74778271,  0.17432562],
       [-2.82089068, -0.08210451,  0.26425109],
       [-2.62648199,  0.17040535, -0.01580151],
       [-2.88795857, -0.57079803,  0.02733541],
       [-2.67384469, -0.1066917 , -0.1915333 ]])